# This notebook uses the twint library to scrape tweets from NBA media accounts for MVP candidates from 2008-2021.

In [ ]:
# This cell is used for google colab use of twint
import os
from google.colab import output
!pip install twint
!pip uninstall twint -y
!git clone --depth=1 https://github.com/twintproject/twint.git
%cd twint/twint/
!sed -i 's/tweet_count = 100/tweet_count = 100 if not config.Limit else config.Limit/g' url.py
%cd ..
!pip3 install . -r requirements.txt
%cd ..
output.clear()

In [ ]:
# non-colab use
#!pip install --user --upgrade git+https://github.com/twintproject/twint.git@origin/master#egg=twint

Import Libraries

In [ ]:
import nest_asyncio
import twint
import pandas as pd

In [ ]:
#Twitter accounts that will be searched
outlet_list= ['espn' , 'ESPNNBA' , 'TurnerSportsPR' , 'TheAthletic' , 'ringer' , 'CBSSports' , 'AP_Sports' , 'BleacherReport' , 'FOXSports' , 'SInow' ,
              'nypostsports', 'siriusxmsports', 'YahooSports', ]

#All MVP Candidates 
mvps_2008_2009 = ['2008-2009','LeBron James', 'Kobe Bryant', 'Dwyane Wade', 'Dwight Howard', 'Chris Paul', 'Chauncey Billups', 'Paul Pierce', 'Tony Parker', 'Brandon Roy', 'Dirk Nowitzki', 'Tim Duncan', 'Yao Ming']
mvps_2009_2010 = ['2009-2010','LeBron James', 'Kevin Durant', 'Kobe Bryant', 'Dwight Howard', 'Dwyane Wade', 'Carmelo Anthony', 'Dirk Nowitzki', 'Steve Nash', 'Deron Williams', "Amar'e Stoudemire", 'Manu Ginobili', 'Chauncey Billups', 'Chris Bosh', 'stephen Jackson', 'Joe Johnson']
mvps_2012_2013 = ['2012-2013','LeBron James', 'Kevin Durant', 'Carmelo Anthony', 'Chris Paul', 'Kobe Bryant', 'Tony Parker', 'Tim Duncan', 'James Harden', 'Russell Westbrook', 'Dwyane Wade', 'steph Curry', 'Kevin Garnett', 'Marc Gasol', 'Ty Lawson', 'David Lee', 'Joakim Noah']
mvps_2015_2016 = ['2015-2016','steph Curry', 'Kawhi Leonard', 'LeBron James', 'Russell Westbrook', 'Kevin Durant', 'Chris Paul', 'Draymond Green', 'Damian Lillard', 'James Harden', 'Kyle Lowry']
mvps_2013_2014 = ['2013-2014','Kevin Durant', 'LeBron James', 'Blake Griffin', 'Joakim Noah', 'James Harden', 'steph Curry', 'Chris Paul', 'Al Jefferson', 'Paul George', 'LaMarcus Aldridge', 'Kevin Love', 'Tim Duncan', 'Tony Parker', 'Dirk Nowitzki', 'Carmelo Anthony', 'Goran Dragic', 'Mike Conley']
mvps_2018_2019 = ['2018-2019','Giannis Antetokounmpo', 'James Harden', 'Paul George', 'Nikola Jokić', 'steph Curry', 'Damian Lillard', 'Joel Embiid', 'Kevin Durant', 'Kawhi Leonard', 'Russell Westbrook', 'Rudy Gobert', 'LeBron James']
mvps_2014_2015 = ['2014-2015','steph Curry', 'James Harden', 'LeBron James', 'Russell Westbrook', 'Anthony Davis', 'Chris Paul', 'LaMarcus Aldridge', 'Marc Gasol', 'Blake Griffin', 'Tim Duncan', 'Kawhi Leonard', 'Klay Thompson']
mvps_2016_2017 = ['2016-2017','Russell Westbrook', 'James Harden', 'Kawhi Leonard', 'LeBron James', 'Isaiah Thomas', 'steph Curry', 'Giannis Antetokounmpo', 'John Wall', 'Anthony Davis', 'Kevin Durant', 'DeMar DeRozan']
mvps_2007_2008 =['2007-2008','Kobe Bryant', 'Chris Paul', 'Kevin Garnett', 'LeBron James', 'Dwight Howard', "Amar'e Stoudemire", 'Tim Duncan', 'Tracy McGrady', 'Steve Nash', 'Manu Ginobili', 'Dirk Nowitzki', 'Deron Williams', 'Carmelo Anthony', 'Carlos Boozer', 'Antawn Jamison', 'Paul Pierce', 'Rasheed Wallace']
mvps_2019_2020 = ['2019-2020','Giannis Antetokounmpo', 'LeBron James', 'James Harden', 'Luka Dončić', 'Kawhi Leonard', 'Anthony Davis', 'Chris Paul', 'Damian Lillard', 'Nikola Jokić', 'Pascal Siakam', 'Jimmy Butler', 'Jayson Tatum']
mvps_2011_2012 = ['2011-2012','LeBron James', 'Kevin Durant', 'Chris Paul', 'Kobe Bryant', 'Tony Parker', 'Kevin Love', 'Dwight Howard', 'Rajon Rondo', 'Steve Nash', 'Dwyane Wade', 'Derrick Rose', 'Dirk Nowitzki', 'Russell Westbrook', 'Tim Duncan', 'Joe Johnson']
mvps_2006_2007 = ['2006-2007','Dirk Nowitzki', 'Steve Nash', 'Kobe Bryant', 'Tim Duncan', 'LeBron James', 'Tracy McGrady', 'Chris Bosh', 'Gilbert Arenas', 'Carlos Boozer', 'Kevin Garnett', 'Chauncey Billups', "Shaquille O'Neal", 'Dwyane Wade', "Amar'e Stoudemire", 'Carmelo Anthony', 'Baron Davis', 'Tony Parker']
mvps_2010_2011 = ['2010-2011','Derrick Rose', 'Dwight Howard', 'LeBron James', 'Kobe Bryant', 'Kevin Durant', 'Dirk Nowitzki', 'Dwyane Wade', 'Manu Ginobili', "Amar'e Stoudemire", 'Blake Griffin', 'Rajon Rondo', 'Tony Parker', 'Chris Paul']
mvps_2017_2018 = ['2017-2018','James Harden', 'LeBron James', 'Anthony Davis', 'Damian Lillard', 'Russell Westbrook', 'Giannis Antetokounmpo', 'Kevin Durant','DeMar DeRozan', 'LaMarcus Aldridge','Jimmy Butler', 'steph Curry']
mvps_2020_2021 = ['2020-2021','Nikola Jokic', 'Joel Embiid', 'Steph Curry', 'Giannis Antetokounmpo' , 'Chris Paul' , 'Luka Dončić' , 'Damian Lillard' , 'Julius Randle' , 'Derrick Rose' , 'Rudy Gobert' , 'Russell Westbrook' , 'Ben Simmons' , 'James Harden' , 'LeBron James' , 'Kawhi Leonard']

In [ ]:
all_mvps = [mvps_2006_2007,mvps_2007_2008,mvps_2008_2009, mvps_2009_2010, mvps_2010_2011, mvps_2011_2012, mvps_2012_2013, mvps_2013_2014,
            mvps_2014_2015, mvps_2015_2016, mvps_2016_2017, mvps_2017_2018, mvps_2018_2019, mvps_2019_2020, mvps_2020_2021]

In [ ]:
# Twint runs into errors when scraping thousands of tweets at a time
# This cell only analyzes a single season
print(all_mvps[1])
mvp_season_to_scrape = all_mvps[1] #change index as needed
nest_asyncio.apply() # needed for twint to work in the runtime

# Getting season range
year_range = mvp_season_to_scrape[0]
start_year = str(year_range)[0:4]
end_year = str(year_range)[5:]
mvp_season_to_scrape.pop(0)

collected_tweets = False
for player in mvp_season_to_scrape:
  for username in outlet_list:
    # Instantiate and configure the twint-object
    c = twint.Config()
    c.Username = username
    c.Store_object = True
    c.Pandas =True
    c.Search = player
    c.Limit = 50000
    c.Hide_output = False

    # lockout season
    if start_year == '2011':
      c.Since = start_year +'-12-25'
      c.Until = end_year + '-04-28'
    # covid seasons
    elif start_year == '2019':
      c.Since = start_year +'-09-28'
      c.Until = end_year + '-08-17'    
    elif start_year == '2020':
      c.Since = start_year +'-12-11'
      c.Until = end_year + '-05-22' 
    # Normal Seasons
    else:
      c.Since = start_year +'-09-28'
      c.Until = end_year + '-04-13'

    c.Lang = 'en'
    twint.run.Search(c)

    # Adding Extra metadata
    twint.storage.panda.Tweets_df['player'] = player
    twint.storage.panda.Tweets_df['season'] = year_range
    twint.storage.panda.Tweets_df['tweet_username'] = username
    twint.storage.panda.Tweets_df['media_type'] = 'outlet' #media
    twint.storage.panda.Tweets_df.to_csv('mvp_tweets_' + str(year_range) + '_' + str(player) + '_' + str(username) + '.csv')

    # This code is used when running this cell multiple times to create a single df
    tempdf = twint.storage.panda.Tweets_df.copy()
    if (('id' in tempdf.columns) & (collected_tweets == False)):
      fulldf = twint.storage.panda.Tweets_df.copy()
      collected_tweets = True
    elif collected_tweets == False:
      pass
    else:
      fulldf = pd.concat([fulldf,tempdf])
  
    print(username + ' ' + player + ' scraped')


In [ ]:
fulldf